In [52]:
import librosa as lb

In [8]:
import numpy as np

In [9]:
from IPython.display import clear_output

In [105]:
from random import randint

In [54]:
import tensorflow as tf

In [55]:
#Initialize random biases
def create_biases(number_of_neurons, n_classes):
    biases = []
    #Hidden layers
    for i in range(len(number_of_neurons)):
        biases.append(tf.Variable(tf.random_normal([number_of_neurons[i]])))
    #Output Layer
    biases.append(tf.Variable(tf.random_normal([n_classes])))
    return biases

In [56]:
#Initialize random weights
def create_weights(number_of_neurons, n_inputs, n_classes):
    weights = []
    #First layer (nºinputs x nºneurons)
    weights.append(tf.Variable(tf.random_normal([n_inputs,number_of_neurons[0]])))
    
    #Hidden layers
    for i in range(1,len(number_of_neurons)):
        weights.append(tf.Variable(tf.random_normal([number_of_neurons[i-1],number_of_neurons[i]])))
    #Output layer (nºneuros x nºoutputs)
    weights.append(tf.Variable(tf.random_normal([number_of_neurons[len(number_of_neurons)-1],
                                                                                  n_classes])))    
    return weights

In [88]:
def multilayer_perceptron(x, weights, biases):
    progression = []
    activation = []
    progression.append(tf.add(tf.matmul(x, weights["w0"]), biases["b0"]))
    activation.append(tf.nn.tanh(progression[0]))
    for i in range(1, len(number_of_neurons)):
        progression.append(tf.add(tf.matmul(activation[i-1], weights[i]), biases[i]))
        activation.append(tf.nn.tanh(progression[i]))
    
    progression.append(tf.add(tf.matmul(activation[len(number_of_neurons)-1],
                                        weights[len(number_of_neurons)]), biases[len(number_of_neurons)]))
    activation.append(tf.nn.tanh(progression[len(number_of_neurons)]))
    
    return activation[len(number_of_neurons)]

In [99]:
learning_rate = 0.003
training_epochs = 1500
batch_size = 1
n_inputs = 13*5    #We get a matrix (13 x 5) when calculating the MFCC
n_classes = 10     #Rock, blues, jazz...

In [102]:
#Input and correct output
x = tf.placeholder("float", [None, n_inputs])
y = tf.placeholder("float", [None, n_classes])

#Create our Neural Network
number_of_neurons = [256, 256, 256,256, 256, 256,256, 256, 256,256, 256, 256]
biases = create_biases(number_of_neurons, n_classes)
weights = create_weights(number_of_neurons, n_inputs, n_classes)
#Get predictions
pred = multilayer_perceptron(x, weights, biases)
n_samples = 1500 # Number of samples we have

In [103]:
#Set cost and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [119]:
#Inizialize variables for TensorFlow
init = tf.global_variables_initializer()
#Start session
sess = tf.InteractiveSession()
sess.run(init)

In [6]:
def getRandomData():
    in_x = np.random.randn(n_samples, n_inputs)
    in_y = np.zeros((n_samples, n_classes))
    for i in range(n_samples):
        matrix[i][randint(0, n_classes-1)] = 1
    return in_x, in_y

In [ ]:
in_x, in_y = getRandomData()

In [107]:
def taking_batches(input_x, expected_y, batch_size, offset):
    return input_x[offset:offset+batch_size],expected_y[offset:offset+batch_size]

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0.0
    total_batches = int(n_samples/batch_size)
    for i in range(total_batches): 
        batch_x,batch_y = taking_batches(in_x,in_y,batch_size,i)
        _,c = sess.run([optimizer,cost], feed_dict={x:batch_x,y:batch_y}) 
        avg_cost += c/total_batches;
        clear_output()
        print("Epoch: {} cost= {:.4f}".format(epoch+1,avg_cost))

print("Finished with {} epochs with cost {}".format(training_epochs,avg_cost))

In [122]:
# Test error
correct_predictions = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
correct_predictions = tf.cast(correct_predictions, "float")
accuracy = tf.reduce_mean(correct_predictions)
print("Accuracy: {}".format(accuracy.eval({x:in_x,y:matrix})))